In [44]:
#Load libraries
import numpy as np
from sklearn.decomposition import PCA
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
#Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
import sklearn.metrics
from sklearn.model_selection import GridSearchCV
from sklearn import decomposition

In [29]:
import pickle
with open(r"training_set.p", "rb") as input_file:
    data_full = pickle.load(input_file)

In [31]:
data_full

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,

In [33]:
# Input data here
y_full = data_full[1]
x_full = data_full[0]
#Label and x-data
#y_full =  np.random.binomial(1, 0.5, size=600)
#x_full = np.random.rand(600,750)
pd.DataFrame(y_full).describe()
#pd.DataFrame(x_full).describe()

,0
count,863.000000
mean,0.492468
std,0.500233
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [46]:
#Try PCA first
pca_all = decomposition.PCA(n_components = 100) #use 150 if you want more variance explained
pca_all.fit(x_full)


PCA(n_components=100)

In [48]:
#get new X
x_full = pca_all.transform(x_full)

ValueError: X has 100 features, but PCA is expecting 2048 features as input.

In [49]:
pd.DataFrame(x_full).describe()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
count,863.000000,863.000000,8.630000e+02,863.000000,8.630000e+02,8.630000e+02,863.000000,8.630000e+02,8.630000e+02,8.630000e+02,...,8.630000e+02,8.630000e+02,863.000000,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02,8.630000e+02
mean,0.000004,-0.000005,-6.077878e-08,-0.000003,-3.978247e-07,9.514642e-07,-0.000001,4.271091e-07,-4.798761e-07,6.812749e-07,...,7.062770e-07,1.801262e-07,0.000001,-3.049990e-07,6.539244e-07,-2.997499e-07,4.307351e-07,1.534664e-07,-1.887250e-07,-3.051716e-07
std,2.903463,1.271854,1.227888e+00,1.146520,9.209772e-01,8.445960e-01,0.762596,6.650893e-01,6.480258e-01,5.509889e-01,...,8.149730e-02,7.994579e-02,0.079575,7.852906e-02,7.796175e-02,7.783684e-02,7.537352e-02,7.462294e-02,7.410250e-02,7.296871e-02
min,-5.924027,-3.423483,-4.489536e+00,-2.888275,-3.092030e+00,-3.133662e+00,-2.651993,-2.150130e+00,-1.800069e+00,-1.475133e+00,...,-2.921217e-01,-3.379734e-01,-0.379355,-2.624988e-01,-2.637689e-01,-3.544321e-01,-2.349452e-01,-2.154722e-01,-2.640032e-01,-2.601485e-01
25%,-2.247991,-0.767917,-8.127637e-01,-0.778933,-6.250587e-01,-5.001075e-01,-0.469976,-4.550333e-01,-4.141287e-01,-3.813930e-01,...,-4.899773e-02,-4.895527e-02,-0.045104,-4.879159e-02,-5.480057e-02,-4.909454e-02,-4.719900e-02,-5.022101e-02,-4.712421e-02,-4.246395e-02
50%,-0.179334,-0.083923,8.521759e-02,-0.130934,-3.642700e-02,1.840365e-02,-0.042228,-5.638916e-02,-3.283519e-03,-3.306805e-02,...,1.767911e-04,2.570747e-03,-0.002666,-9.439141e-05,1.129597e-03,2.173834e-03,-2.011247e-04,-6.277151e-04,1.706622e-03,-1.331348e-03
75%,2.028249,0.671135,8.434432e-01,0.634597,6.103601e-01,5.436060e-01,0.455045,3.840629e-01,3.983438e-01,3.010271e-01,...,5.012901e-02,4.551029e-02,0.047702,5.039289e-02,5.200769e-02,4.660066e-02,4.672504e-02,4.919171e-02,4.632358e-02,4.502040e-02
max,9.400922,5.528552,4.758934e+00,5.382077,3.156904e+00,3.347639e+00,3.713262,2.702805e+00,3.812306e+00,2.124125e+00,...,4.033509e-01,5.086499e-01,0.394754,2.900116e-01,2.848226e-01,4.684196e-01,2.550550e-01,3.347202e-01,2.812885e-01,2.715169e-01


In [50]:
#Try Random forest and other classifiers
rf_split = RandomForestClassifier(random_state=1337,criterion="entropy",max_depth=10, min_samples_split=15,n_estimators=1000) 
#rf_split.fit(x_full,y_full)
#Support Vector Classifier, C should actually be seperately tuned 
svm_split = sklearn.svm.SVC(random_state=1337,C=0.5) 
#Ridge Classifier, alpha should actually be seperately tuned
ridge_split = sklearn.linear_model.RidgeClassifier(random_state=1337,normalize=True,alpha=2)
#Ada Boost 
ada_split = sklearn.ensemble.AdaBoostClassifier(n_estimators=1000,random_state=1337)

In [51]:
scores_rf=cross_val_score(rf_split, x_full, y_full, cv=10,n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_rf

array([0.62068966, 0.62068966, 0.5862069 , 0.6744186 , 0.62790698,
       0.74418605, 0.65116279, 0.60465116, 0.65116279, 0.70930233])

In [52]:
scores_svm=cross_val_score(svm_split, x_full, y_full, cv=10,n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_svm

array([0.67816092, 0.55172414, 0.5862069 , 0.6744186 , 0.65116279,
       0.68604651, 0.55813953, 0.69767442, 0.65116279, 0.69767442])

In [53]:
scores_ridge=cross_val_score(ridge_split, x_full, y_full, cv=10,n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_ridge

array([0.6091954 , 0.62068966, 0.64367816, 0.68604651, 0.6627907 ,
       0.72093023, 0.6744186 , 0.62790698, 0.68604651, 0.6627907 ])

In [54]:
scores_ada=cross_val_score(ada_split, x_full, y_full, cv=10,n_jobs=-1,scoring=make_scorer(accuracy_score))
scores_ada

array([0.52873563, 0.57471264, 0.55172414, 0.58139535, 0.52325581,
       0.6744186 , 0.60465116, 0.60465116, 0.61627907, 0.58139535])

In [55]:
print("Mean 10-fold CV Accuracy: ")
print("Random Forest: ",np.mean(scores_rf))
print("Support Vector Machine: ", np.mean(scores_svm))
print("Ridge Classifier: ",np.mean(scores_ridge))
print("Ada Boost: ", np.mean(scores_ada))

Mean 10-fold CV Accuracy: 
Random Forest:  0.6490376904570971
Support Vector Machine:  0.6432371023790431
Ridge Classifier:  0.6594493450948945
Ada Boost:  0.584121892542101
